# COPERNICUS MARINE DATASET - THE NORTH SEA

# Feature importance analysis (using XGBoost)

https://help.marine.copernicus.eu/en/articles/8283072-copernicus-marine-toolbox-api-subset

https://pypi.org/project/copernicusmarine/

In [ ]:
import copernicusmarine
import os
import pandas as pd
from tqdm import tqdm
import numpy as np
#import shap
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# To avoid warning messages
import warnings
warnings.filterwarnings('ignore')

# Import Garbage Collector - we will need it a lot here, since we are dealing with huge files and might have memory issues!
import gc

In [ ]:
# You need to log in a Copernicus Marine account to access the data.
copernicusmarine.login()
# Copernicus username and password.

# Import datasets as pandas dataframes (skip this if you already have merged_df.csv)

(Based on Kshitiz's code)

In [ ]:
NORTH_SEA_REGION_lat = [50, 62]
NORTH_SEA_REGION_lon = [-6, 12]
TIMEFRAME = ["1997-01-01T00:00:00", "2023-01-01T00:00:00"]
DEPTH = [0, 0.5]

TIMEFRAMES_SST = [["1997-01-01T00:00:00", "2002-12-01T00:00:00"], ["2002-12-02T00:00:00", "2007-12-01T00:00:00"], ["2007-12-02T00:00:00", "2012-12-01T00:00:00"],
                  ["2012-12-02T00:00:00", "2017-12-01T00:00:00"], ["2017-12-02T00:00:00", "2023-01-01T00:00:00"]]

SST_COUNTER = 0

CURR_DIR = os.getcwd()
DATASETS_DIR = CURR_DIR + "\\datasets_csv"
os.makedirs(DATASETS_DIR, exist_ok = True)

In [ ]:
def get_and_polish_dataset(dataset_id, variables, output_name, SST_COUNTER):
    HAS_DEPTH = {'bathy', 'carbon', 'chlorophyll', 'pisces'}
    
    if output_name in HAS_DEPTH:
        depth_ = DEPTH
    else:
        depth_ = [None, None]


    if (not output_name == 'bathy') and (not output_name == 'sst'):
        time_ = TIMEFRAME
    elif output_name == 'sst':
        time_ = TIMEFRAMES_SST[SST_COUNTER]
        output_name = output_name + '_' + str(SST_COUNTER)
    else:
        time_ = [None, None]

    data_request = {
    "dataset_id" : dataset_id,
    "variables" : variables,
    "longitude" : NORTH_SEA_REGION_lon, 
    "latitude" : NORTH_SEA_REGION_lat,
    "time" : time_,
    "depth": depth_
    } 

    if not output_name == 'bathy':
        df = copernicusmarine.read_dataframe(dataset_id=data_request["dataset_id"],
                                        variables=data_request["variables"],
                                        minimum_longitude=data_request["longitude"][0],
                                        maximum_longitude=data_request["longitude"][1],
                                        minimum_latitude=data_request["latitude"][0],
                                        maximum_latitude=data_request["latitude"][1],
                                        minimum_depth=data_request["depth"][0],
                                        maximum_depth=data_request["depth"][1],
                                        start_datetime=data_request["time"][0],
                                        end_datetime=data_request["time"][1]
                                        )
    else:
         df = copernicusmarine.read_dataframe(dataset_id=data_request["dataset_id"],
                                        variables=data_request["variables"],
                                        minimum_longitude=data_request["longitude"][0],
                                        maximum_longitude=data_request["longitude"][1],
                                        minimum_latitude=data_request["latitude"][0],
                                        maximum_latitude=data_request["latitude"][1],
                                        minimum_depth=data_request["depth"][0],
                                        maximum_depth=data_request["depth"][1]
                                        )

    df = df.dropna()


    # function to coarse grain the data and make resolution same as Pisces data
    def coarse_grain(df, features, output_name):
        """
            Parameters
            ----------
            df: pandas dataframe containing the data accessed from copernicus mariner
            features: name of the features in the dataframe

            Output
            ------
            a pandas dataframe with feature values for 0.25 deg x 0.25 deg resolution 

            """
        
        df["0_1"] = df["latitude"].to_numpy() - np.floor(df["latitude"])
        conditions = [df["0_1"] < 0.25,
                        (df["0_1"] >= 0.25)  & (df["0_1"] < 0.5),
                        (df["0_1"] >= 0.5)  & (df["0_1"] < 0.75),
                        (df["0_1"] >= 0.75)  & (df["0_1"] < 1)]
        outputs = [0, 0.25, 0.5, 0.75]
        df['latitude'] = np.floor(df["latitude"]) + np.select(conditions, outputs)
        df = df.drop(columns=["0_1"])

        df["0_1"] = df["longitude"].to_numpy() - np.floor(df["longitude"])
        # if not redefined then conditions is based on latitude
        conditions = [df["0_1"] < 0.25,
                    (df["0_1"] >= 0.25)  & (df["0_1"] < 0.5),
                    (df["0_1"] >= 0.5)  & (df["0_1"] < 0.75),
                    (df["0_1"] >= 0.75)  & (df["0_1"] < 1)]
        df['longitude'] = np.floor(df["longitude"]) + np.select(conditions, outputs)
        df = df.drop(columns=["0_1"])

        if not output_name == 'bathy': 
            return df.groupby(["time", "latitude", "longitude"])[features].mean()
        else:
            return df.groupby(["latitude", "longitude"])[features].mean()
    
    if not output_name == 'pisces':
        df = df.reset_index()
        if not output_name == 'bathy':
            df_cg = coarse_grain(df, df.columns[3:].tolist(), output_name)
        else:
            df_cg = coarse_grain(df, df.columns[2:].tolist(), output_name)
    else:
        # remove depth as index
        df = df.reset_index(level=["depth"])
        # average across the depth for each (time, latitude, longitude)
        df_cg = df.reset_index().groupby(["time","latitude","longitude"]).mean()
        df_cg = df_cg.drop(columns=["depth"])


    # remove time and just have month and year
    df_cg = df_cg.reset_index()

    if not output_name == 'bathy':
        # from https://stackoverflow.com/questions/53509168/extract-year-month-and-day-from-datetime64ns-utc-python
        datetimes = pd.to_datetime(df_cg['time'])
        df_cg['day'] = datetimes.dt.day
        df_cg['month'] = datetimes.dt.month
        df_cg['year'] = datetimes.dt.year

        # remove the time column
        df_cg = df_cg.drop(columns=["time"])
        df_cg = df_cg.set_index(["year","month","day","latitude","longitude"])
        df_cg = df_cg.groupby(["year","month","latitude","longitude"]).mean()
        try:
             df_cg = df_cg.drop(columns = ["day"])
        except: 
            pass

    df_cg.to_csv(f"{DATASETS_DIR}/{output_name}_049depth.csv")

    del df, df_cg
    gc.collect()

    return SST_COUNTER

In [ ]:
bathy_info = { 'dataset_id': 'cmems_mod_glo_phy_my_0.083deg_static',
               'variables': ['deptho'],
               'output_name' : 'bathy'}

carbon_info = { 'dataset_id': 'dataset-carbon-rep-monthly',
               'variables': ["fgco2", "omega_ar", "omega_ca", "ph", "spco2", "talk", "tco2"],
               'output_name' : 'carbon'}

chlorophyll_info = { 'dataset_id': 'cmems_obs-oc_glo_bgc-plankton_my_l4-multi-4km_P1M',
               'variables': ["CHL"],
               'output_name' : 'chlorophyll'}

pisces_info = { 'dataset_id': 'cmems_mod_glo_bgc_my_0.25deg_P1M-m',
               'variables': ["fe", "no3", "o2", "po4", "si"],
               'output_name' : 'pisces'}

sssd_info = { 'dataset_id': 'cmems_obs-mob_glo_phy-sss_my_multi_P1M',
               'variables': ["sos","dos"],
               'output_name' : 'sssd'}

sst_info = { 'dataset_id': 'METOFFICE-GLO-SST-L4-REP-OBS-SST',
               'variables': ["analysed_sst"],
               'output_name' : 'sst'}

In [ ]:
dataset_names = [bathy_info, carbon_info, chlorophyll_info, pisces_info, sst_info]
dataset_names = [carbon_info, chlorophyll_info, pisces_info, sst_info]
# For sssd, use Kshitiz's notebook directly.

In [ ]:
for name in tqdm(dataset_names):
    if not name['output_name'] == 'sst':
        get_and_polish_dataset(name['dataset_id'], name['variables'], name['output_name'], SST_COUNTER)
    else:
        for _ in range(len(TIMEFRAMES_SST)):
            get_and_polish_dataset(name['dataset_id'], name['variables'], name['output_name'], SST_COUNTER)
            SST_COUNTER += 1


### Merge datasets together

In [ ]:
df_sst = [pd.read_csv(f"{DATASETS_DIR}/sst_{i}_049depth.csv") for i in range(len(TIMEFRAMES_SST))]
df_sst = pd.concat(df_sst)

In [ ]:
df_sst.head()

In [ ]:
df_sst.to_csv(f"{DATASETS_DIR}/sst_049depth.csv")

In [ ]:
all_df = []
for df in os.listdir(DATASETS_DIR):
    if not df.startswith('merged'):
        if not df.startswith('bathy'):
            all_df.append(pd.read_csv(f"{DATASETS_DIR}/{df}").set_index(["year", "month", "latitude", "longitude"]))
        #else:
        #    all_df.append(pd.read_csv(f"{DATASETS_DIR}/{df}").set_index(["latitude", "longitude"]))
[len(x) for x in all_df]

In [ ]:
# Some datasets still have an unnamed column corresponding to an indexing column that we need to drop
cleaned_all_df = []
for df in all_df:
    try:
        cleaned_all_df.append(df.drop(columns=['Unnamed: 0']))
    except:
        cleaned_all_df.append(df)

In [ ]:
merged_df = cleaned_all_df[0]
for df in cleaned_all_df[1:]:
    merged_df = merged_df.join(df, how = 'inner')

In [ ]:
merged_df.tail(5)

In [ ]:
len(merged_df)

In [ ]:
merged_df.to_csv(f"{DATASETS_DIR}/merged_df_049depth.csv")

In [ ]:
del cleaned_all_df, all_df, df_sst
gc.collect()

# Retrieve merged dataframe (if you already had created merged_df.csv)

In [ ]:
CURR_DIR = os.getcwd()
DATASETS_DIR = CURR_DIR + "\\datasets_csv"

In [ ]:
merged_df = pd.read_csv(f"{DATASETS_DIR}\merged_df_049depth.csv")
len(merged_df)

In [ ]:
# Get rid of outliers for chlorophyll values
y = merged_df[['CHL']]
print("Min, max, average, std, 99th percentile values of chlorophyll in the merged dataframe:", 
      [y.min(axis=0)['CHL'], y.max(axis=0)['CHL'], y.median(axis=0)['CHL'], y.std()[0], np.percentile(y, 99)])

style = {'facecolor': 'none', 'edgecolor': 'C0', 'linewidth': 3}
fig, ax = plt.subplots()
ax.hist(y, bins=100, **style)
plt.show()

In [ ]:
# Get rid of values above the 99th percentile value
chl_threshold =  np.percentile(y, 99)
merged_df_chl_filtered = merged_df[merged_df["CHL"] < chl_threshold]
len(merged_df_chl_filtered)

In [ ]:
merged_df_original = merged_df.copy()
merged_df = merged_df_chl_filtered

In [ ]:
# Get rid of outliers for chlorophyll values
y = merged_df[['CHL']]
print("Min, max, average, std, 99th percentile values of chlorophyll in the merged filtered dataframe:", 
      [y.min(axis=0)['CHL'], y.max(axis=0)['CHL'], y.median(axis=0)['CHL'], y.std()[0], np.percentile(y, 99)])

style = {'facecolor': 'none', 'edgecolor': 'C0', 'linewidth': 3}
fig, ax = plt.subplots()
ax.hist(y, bins=100, **style)
plt.show()

# Split data into train, val, test sets

In [ ]:
# Extract feature and target arrays
X = merged_df.drop('CHL', axis = 1)
y = merged_df[['CHL']]
#X = X.drop('time', axis = 1)
#X = X.drop('latitude', axis = 1)
#X = X.drop('longitude', axis = 1)

del merged_df
gc.collect()

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1729)
X_train_train, X_train_test, y_train_train, y_train_test = train_test_split(X_train, y_train, random_state = 1729)

del X_train, y_train
gc.collect()

In [ ]:
# Save the train, val, and test sets
X_train_train.to_csv((f"{DATASETS_DIR}/X_train_train_049depth.csv"))
y_train_train.to_csv((f"{DATASETS_DIR}/y_train_train_049depth.csv"))
X_train_test.to_csv((f"{DATASETS_DIR}/X_train_test_049depth.csv"))
y_train_test.to_csv((f"{DATASETS_DIR}/y_train_test_049depth.csv"))
X_test.to_csv((f"{DATASETS_DIR}/X_test_049depth.csv"))
y_test.to_csv((f"{DATASETS_DIR}/y_test_049depth.csv"))